In [20]:
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\clint\Desktop\Lifecycle_RA\Data\Processed\Combined_Csvs\combined_data.csv"
data = pd.read_csv(file_path)

# Display the first few rows of the data
data

,Date,4YO,5YO,Source_File,3YO,2YO,Quarter,Month,2YO_no_outliers,3YO_no_outliers,4YO_no_outliers,5YO_no_outliers
0,2015-01-01,79700.0,69700.0,02_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,79700.0,69700.0
1,2015-01-01,78000.0,70000.0,03_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
2,2015-01-01,78000.0,70000.0,04_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
3,2015-01-01,78000.0,70000.0,08_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
4,2015-01-01,78000.0,70000.0,05_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2115,2024-09-01,54000.0,41000.0,03_2025.csv,93000.0,NaN,2024Q3,2024-09,NaN,93000.0,54000.0,41000.0
2116,2024-09-01,73000.0,49000.0,01_2025.csv,94000.0,NaN,2024Q3,2024-09,NaN,94000.0,73000.0,49000.0
2117,2024-11-01,51000.0,43000.0,03_2025.csv,92000.0,NaN,2024Q4,2024-11,NaN,92000.0,51000.0,43000.0
2118,2024-11-01,58000.0,44000.0,01_2025.csv,84000.0,NaN,2024Q4,2024-11,NaN,84000.0,58000.0,44000.0


In [21]:
import pandas as pd
import numpy as np

# First, let's convert the date column to datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Create a month column
data['Month'] = data['Date'].dt.to_period('M')

# Create a quarter column
data['Quarter'] = data['Date'].dt.to_period('Q')

# Convert string columns to numeric
numeric_columns = ['2YO', '3YO', '4YO', '5YO', '3-5YO Avg.', '3-5YO Avg']
for col in numeric_columns:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

# Drop columns '3-5YO Avg.', '3-5YO Avg'
data.drop(columns=['3-5YO Avg.', '3-5YO Avg'], inplace=True, errors='ignore')

# Save data back to the file
data.to_csv(file_path, index=False)

# Display data types after conversion
print(data.dtypes)

# Define functions for statistics without extremes
def mean_no_extremes(x):
    if len(x.dropna()) <= 2:
        return np.nan
    return x.dropna()[(x.dropna() != x.dropna().min()) & (x.dropna() != x.dropna().max())].mean()

def median_no_extremes(x):
    if len(x.dropna()) <= 2:
        return np.nan
    return x.dropna()[(x.dropna() != x.dropna().min()) & (x.dropna() != x.dropna().max())].median()

# Define a function to remove outliers beyond 2 standard deviations
def remove_outliers_2std(x):
    if len(x.dropna()) == 0:
        return x
    mean = x.mean()
    std = x.std()
    return x[(x >= mean - 2 * std) & (x <= mean + 2 * std)]

# Create new columns for each numeric column excluding outliers
for column in ['2YO', '3YO', '4YO', '5YO']:
    if column in data.columns:
        data[f'{column}_no_outliers'] = remove_outliers_2std(data[column])

# Create statistics for all columns
results = {}

# Updated to include all requested columns
for column in ['2YO', '3YO', '4YO', '5YO']:
    if column in data.columns:
        # Calculate statistics for the current column
        quarterly_stats = data.groupby('Quarter')[column].agg([
            ('mean', 'mean'),
            ('median', 'median'),
            ('min', 'min'),
            ('max', 'max'),
            ('mean_no_extremes', mean_no_extremes),
            ('median_no_extremes', median_no_extremes)          
        ])
        
        # Rename columns to include the original column name
        quarterly_stats.columns = [f'{column}_{stat}' for stat in quarterly_stats.columns]
        
        # Store in dictionary
        results[column] = quarterly_stats

        # Calculate statistics for the no_outliers column
        if f'{column}_no_outliers' in data.columns:
            no_outliers_stats = data.groupby('Quarter')[f'{column}_no_outliers'].agg([
                ('mean', 'mean'),
                ('median', 'median'),
                ('min', 'min'),
                ('max', 'max'),
                ('mean_no_extremes', mean_no_extremes),
                ('median_no_extremes', median_no_extremes)
            ])
            
            # Rename columns to include the original column name
            no_outliers_stats.columns = [f'{column}_no_outliers_{stat}' for stat in no_outliers_stats.columns]
            
            # Merge with the existing stats
            results[column] = pd.concat([results[column], no_outliers_stats], axis=1)

# Combine all results into a single dataframe if results is not empty
if results:
    combined_stats = pd.concat(results.values(), axis=1)
    
    # Reset index to make Quarter a column
    combined_stats = combined_stats.reset_index()
    
    # Convert Quarter period to string for better display
    combined_stats['Quarter'] = combined_stats['Quarter'].astype(str)
    
    # Display the resulting dataframe
    combined_stats
else:
    print("No valid columns were processed. Check your data.")

data

Date               datetime64[ns]
4YO                       float64
5YO                       float64
Source_File                object
3YO                       float64
2YO                       float64
Quarter             period[Q-DEC]
Month                   period[M]
2YO_no_outliers           float64
3YO_no_outliers           float64
4YO_no_outliers           float64
5YO_no_outliers           float64
dtype: object


,Date,4YO,5YO,Source_File,3YO,2YO,Quarter,Month,2YO_no_outliers,3YO_no_outliers,4YO_no_outliers,5YO_no_outliers
0,2015-01-01,79700.0,69700.0,02_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,79700.0,69700.0
1,2015-01-01,78000.0,70000.0,03_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
2,2015-01-01,78000.0,70000.0,04_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
3,2015-01-01,78000.0,70000.0,08_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
4,2015-01-01,78000.0,70000.0,05_2018.csv,NaN,NaN,2015Q1,2015-01,NaN,NaN,78000.0,70000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2115,2024-09-01,54000.0,41000.0,03_2025.csv,93000.0,NaN,2024Q3,2024-09,NaN,93000.0,54000.0,41000.0
2116,2024-09-01,73000.0,49000.0,01_2025.csv,94000.0,NaN,2024Q3,2024-09,NaN,94000.0,73000.0,49000.0
2117,2024-11-01,51000.0,43000.0,03_2025.csv,92000.0,NaN,2024Q4,2024-11,NaN,92000.0,51000.0,43000.0
2118,2024-11-01,58000.0,44000.0,01_2025.csv,84000.0,NaN,2024Q4,2024-11,NaN,84000.0,58000.0,44000.0


In [22]:
output_file_path = r"C:\Users\clint\Desktop\Lifecycle_RA\Data\Processed\Combined_Csvs\combined_stats_quarterly.csv"
combined_stats.to_csv(output_file_path, index=False)